<a href="https://colab.research.google.com/github/Dyuko/Natural_Language_To_Sql/blob/main/tp2_tp_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Consulta SQL con LLM
Este notebook implementa un sistema de consulta en lenguaje natural para una base de datos SQLite (Chinook)
utilizando un modelo de lenguaje (LLM) para convertir preguntas en consultas SQL.

### Instalación de bibliotecas necesarias

In [5]:
!pip install -qU langchain-huggingface
!pip install -qU langchain-core
!pip install faiss-cpu
!pip install python-dotenv
!pip install langchain-community
!pip install langgraph
!pip install -U langchain-groq

### Importación de bibliotecas

In [6]:
import dotenv
import os
import ast
import re
from google.colab import drive
from langchain_community.utilities import SQLDatabase
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_core.messages import HumanMessage

### Montar Google Drive para acceder a archivos

In [7]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/tp2_tpfinal/Chinook_v2.db'

Mounted at /content/drive


### Cargar variables de entorno

In [8]:
dotenv.load_dotenv('/content/drive/MyDrive/.env')
api_key = os.environ.get('GROP_API_KEY')

### Inicializar la conexión a la base de datos SQLite

In [9]:
db = SQLDatabase.from_uri(f'sqlite:///{path}')

### Mostrar información sobre la base de datos

In [10]:
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Ejemplo de consulta a la base de datos

In [11]:
db.run("SELECT * FROM Artist LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### Inicializar el modelo de lenguaje (Llama) con Groq como proveedor

In [12]:
#llm = init_chat_model("llama3-8b-8192", model_provider="groq", api_key=api_key ) #Observación: Supera los tokens disponibles por minuto
llm = init_chat_model("meta-llama/llama-4-scout-17b-16e-instruct", model_provider="groq", api_key=api_key )

### Configurar embeddings

In [13]:
# Configuración del modelo de embeddings para representar texto como vectores
# Se utiliza para búsquedas semánticas de ejemplos similares
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Definir ejemplos few-shot y crear índice FAISS

In [14]:
# Ejemplos de pares (pregunta en lenguaje natural, consulta SQL equivalente)
# Estos ejemplos servirán como referencia para el LLM mediante few-shot learning
examples = [
    {"input": "List all artists.", "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the Artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]

### Convertir ejemplos a textos indexables

In [15]:
# Formateo de los ejemplos para su indexación en FAISS
example_texts = [f"Input: {ex['input']}\nSQL Query: {ex['query']}" for ex in examples]
# Creación del índice FAISS para búsqueda eficiente de ejemplos similares
example_store = FAISS.from_texts(example_texts, embeddings)
# Configuración del recuperador para obtener los 2 ejemplos más similares a una consulta
example_retriever = example_store.as_retriever(search_kwargs={"k": 2})

### System Prompt base

In [16]:
# Prompt de sistema que guía al modelo en cómo responder a las preguntas
# Define un orden específico de pasos y reglas para generar consultas SQL correctas
base_system_message = """Sigue ESTE ORDEN para responder la pregunta:
1. Usa 'sql_db_list_tables' (tablas disponibles)
2. Usa 'sql_db_schema' (estructura de tablas relevantes)
3. Usa 'retrieve_sql_examples' (ejemplos similares)
4. Solo si es necesario: 'search_proper_nouns' (nombres propios)

Reglas clave:
- No utilizar tablas no presentes en sql_db_list_tables y columnas no presentes en sql_db_schema
- Limita resultados a {top_k} (salvo indicación)
- Nunca hagas DML (INSERT/UPDATE/DELETE)
- Verifica consultas antes de ejecutar
- Para nombres propios, SIEMPRE usa la herramienta 'search_proper_nouns'

Ejemplo conciso:
1. Listar tablas → 2. Ver esquema → 3. Buscar ejemplos → [4. Nombres si needed] → 5. Consulta realizada con SQLite
""".format(top_k=5)

### Inicializar el toolkit para SQL con la base de datos y el modelo LLM

In [17]:
# Creación del toolkit que proporciona herramientas para interactuar con la base de datos
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

### Función auxiliar para convertir resultados de consultas SQL en listas limpias

In [18]:
def query_as_list(db, query):
    """
    Convierte el resultado de una consulta SQL en una lista limpia de strings.

    Args:
        db (SQLDatabase): La base de datos donde ejecutar la consulta
        query (str): La consulta SQL a ejecutar

    Returns:
        list: Lista de strings con los resultados limpios (sin duplicados y sin números)
    """
    # Ejecutar la consulta en la base de datos
    res = db.run(query)

    # Convertir el resultado (string) a una estructura de datos Python y aplanar la lista
    res = [el for sub in ast.literal_eval(res) for el in sub if el]

    # Eliminar números y espacios extra de cada string
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]

    # Devolver lista sin duplicados
    return list(set(res))

### Obtener nombres de artistas y álbumes para el sistema de recuperación

In [19]:
# Extracción de nombres de artistas y álbumes de la base de datos
# Estos se usarán para buscar coincidencias cuando se mencionen en las consultas
artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")

### Crear vector store para nombres propios usando FAISS

In [20]:
# Creación de un índice FAISS para la búsqueda eficiente de nombres propios
proper_noun_store = FAISS.from_texts(artists + albums, embeddings)
retriever = proper_noun_store.as_retriever(search_kwargs={"k": 5})

### Configurar herramienta de recuperación para nombres propios

In [21]:
# Descripción de la herramienta para buscar nombres propios
proper_noun_description  = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

# Creación de la herramienta de recuperación para nombres propios
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=proper_noun_description ,
)

### Configurar herramienta de recuperación de ejemplos sql

In [22]:
# Descripción de la herramienta para recuperar ejemplos de consultas SQL
example_description = (
"Use to get examples of similar SQL queries. "
"Input should be a natural language question about the database. "
"Output will be example questions and their corresponding SQL queries."
)

# Creación de la herramienta de recuperación para ejemplos SQL
example_retriever_tool = create_retriever_tool(
    example_retriever,
    name="retrieve_sql_examples",
    description=example_description,
)

### Añadir la herramientas de recuperación a las herramientas disponibles

In [23]:
# Agregar las nuevas herramientas a la lista de herramientas disponibles
tools.extend([retriever_tool, example_retriever_tool])

In [24]:
# Crear agente con el prompt base
agent = create_react_agent(llm, tools, prompt=base_system_message)

### Internal Debug

In [25]:
def answer(question):
    """
    Función para pruebas internas que ejecuta el agente y muestra todos los pasos de razonamiento.

    Args:
        question (str): La pregunta del usuario sobre la base de datos

    Returns:
        None: Imprime cada paso del proceso de razonamiento
    """
    # Crear agente con el prompt actualizado y modo debug activado
    agent = create_react_agent(llm, tools, prompt=base_system_message, debug=True)

    # Ejecutar el agente en modo stream para ver cada paso
    for step in agent.stream(
        {"messages": [{"role": "user", "content": question}]},
        stream_mode="values",
    ):
        step["messages"][-1].pretty_print()  # Imprimir cada mensaje generado

In [26]:
# Ejemplo de prueba interna
#answer("Which country's customers spent the most?")

### Función final para realizar preguntas

In [27]:
def pretty_answer(question):
    """
    Función principal simplificada que recibe una pregunta y devuelve la respuesta del agente.

    Utiliza el método invoke() en lugar de stream() para obtener directamente el resultado final.

    Args:
        question (str): La pregunta del usuario sobre la base de datos

    Returns:
        str: La respuesta generada por el agente (solo el contenido final)
    """
    try:
        # Preparar la entrada y ejecutar el agente de forma directa
        input_data = {"messages": [{"role": "user", "content": question}]}
        result = agent.invoke(input_data)

        # Obtener el último mensaje (respuesta final)
        final_response = result["messages"][-1].content

        return final_response

    except Exception as e:
        return f"Error al procesar la pregunta: {str(e)}"

# Función para imprimir de forma elegante
def print_answer(question):
    """
    Imprime la pregunta y respuesta con formato claro y separación visual

    Args:
        question (str): Pregunta a mostrar
    """
    print("═" * 80)
    print(f"❓ Pregunta: {question}")
    print("─" * 80)
    response = pretty_answer(question)
    print(f"💡 Respuesta:\n{response}")
    print("═" * 80 + "\n")

### Realizar preguntas de prueba

#### Pregunta 1

In [35]:
print_answer("Which country's customers spent the most?")

════════════════════════════════════════════════════════════════════════════════
❓ Pregunta: Which country's customers spent the most?
────────────────────────────────────────────────────────────────────────────────
💡 Respuesta:
The country with the highest total spending is the USA, with a total of $523.06.
════════════════════════════════════════════════════════════════════════════════



In [28]:
# Solución:
db.run("SELECT Country, SUM(Total) AS TotalPurchase FROM Invoice JOIN Customer ON Invoice.CustomerId = Customer.CustomerId GROUP BY Country ORDER BY TotalPurchase DESC LIMIT 1")

"[('USA', 523.0600000000003)]"

#### Pregunta 2

In [36]:
print_answer("List all albums by AC/DC")

════════════════════════════════════════════════════════════════════════════════
❓ Pregunta: List all albums by AC/DC
────────────────────────────────────────────────────────────────────────────────
💡 Respuesta:
The albums by AC/DC are:

1. For Those About To Rock We Salute You
2. Let There Be Rock
════════════════════════════════════════════════════════════════════════════════



In [29]:
# Solución:
db.run("SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC')")

"[(1, 'For Those About To Rock We Salute You', 1), (4, 'Let There Be Rock', 1)]"

#### Pregunta 3

In [39]:
print_answer("What is the average duration of tracks in the Rock genre?")

════════════════════════════════════════════════════════════════════════════════
❓ Pregunta: What is the average duration of tracks in the Rock genre?
────────────────────────────────────────────────────────────────────────────────
💡 Respuesta:
The average duration of tracks in the Rock genre is approximately 283.91 seconds or 4.73 minutes.
════════════════════════════════════════════════════════════════════════════════



In [30]:
# Solución:
db.run("SELECT AVG(Milliseconds) FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock')")

'[(283910.0431765613,)]'

#### Pregunta 4

In [31]:
print_answer("Which artist has the most albums in the database?")

════════════════════════════════════════════════════════════════════════════════
❓ Pregunta: Which artist has the most albums in the database?
────────────────────────────────────────────────────────────────────────────────
💡 Respuesta:
The artist with the most albums in the database is Iron Maiden with 21 albums.
════════════════════════════════════════════════════════════════════════════════



In [32]:
# Solución:
db.run("SELECT ar.Name AS ArtistName, COUNT(al.AlbumId) AS AlbumCount FROM Artist ar JOIN Album al ON ar.ArtistId = al.ArtistId GROUP BY ar.Name ORDER BY AlbumCount DESC LIMIT 1;")

"[('Iron Maiden', 21)]"

#### Pregunta 5

In [33]:
print_answer("Which playlist has the highest number of tracks?")

════════════════════════════════════════════════════════════════════════════════
❓ Pregunta: Which playlist has the highest number of tracks?
────────────────────────────────────────────────────────────────────────────────
💡 Respuesta:
The playlist with the highest number of tracks is "Music".
════════════════════════════════════════════════════════════════════════════════



In [34]:
# Solución:
db.run("SELECT p.Name AS PlaylistName, COUNT(pt.TrackId) AS TrackCount FROM Playlist p JOIN PlaylistTrack pt ON p.PlaylistId = pt.PlaylistId GROUP BY p.Name ORDER BY TrackCount DESC LIMIT 1;")

"[('Music', 6580)]"